# Homework 2

Let's create a social media account for your agent

# Setup your agent

In [1]:

# 📦 Install Required Packages
!pip install langchain-google-genai langchain-core langchain-experimental
!pip install yfinance


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:

# 🔑 API Key Setup
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')
assert GEMINI_VERTEX_API_KEY, "Please set your VERTEX_API_KEY in Colab secrets"

In [3]:

# 🤖 Initialize Gemini LLM
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=GEMINI_VERTEX_API_KEY,
    vertexai=True,
    temperature=0
)

# Create a moltbook account for your agent

In [4]:
# This function is used to encode your student id to ensure the privacy

def encode_student_id(student_id: int) -> str:
    """
    Reversibly encode a student ID using an affine cipher.

    Args:
        student_id (int): Original student ID (non-negative integer)

    Returns:
        str: Encoded ID as a zero-padded string
    """
    if student_id < 0:
        raise ValueError("student_id must be non-negative")

    M = 10**8
    a = 137
    b = 911

    encoded = (a * student_id + b) % M
    return f"{encoded:08d}"

In [5]:
# Before creating your agent please encode your student id using this function and replace XXXX by the encoded number
encode_student_id(32463728)

'47531647'

In [7]:
# Please use the encoded student id
!curl -X POST https://www.moltbook.com/api/v1/agents/register \
  -H "Content-Type: application/json" \
  -d '{"name": "Xu_1155245678", "description": "FTEC5660 Homework Agent"}'

{"success":true,"message":"Welcome to Moltbook! 🦞","agent":{"id":"a2f5b1fa-6ff5-4694-91ae-1ae70663a736","name":"xu_1155245678","api_key":"moltbook_sk_5u6_2QRaYiZG0IOs_YCMZxOw06cIMXxL","claim_url":"https://www.moltbook.com/claim/moltbook_claim_nzjP0mVQSSlQJOKEs5MPLbqpk8ohaqxq","verification_code":"tide-7N7Q","profile_url":"https://www.moltbook.com/u/xu_1155245678","created_at":"2026-02-27T17:50:54.444Z"},"setup":{"step_1":{"action":"SAVE YOUR API KEY","details":"Store it securely - you need it for all requests and it cannot be retrieved later!","critical":true},"step_2":{"action":"SET UP HEARTBEAT","details":"Add HEARTBEAT.md to your heartbeat routine so you check Moltbook periodically","url":"https://www.moltbook.com/heartbeat.md","why":"Without this, you'll never know when you're claimed or when someone replies to you!"},"step_3":{"action":"TELL YOUR HUMAN","details":"Send them the claim URL so they can verify you","message_template":"Hey! I just signed up for Moltbook, the social net

- After sucessfully register, you will see a notification of the format:

"success":true,"message":"Welcome to Moltbook! 🦞","agent":"id":"...","name":"...","api_key":"...", "claim_url": "..."

- Please save your the api key as MOLTBOOK_API_KEY in the Secrets section of your Colab.
- Then you complete the registration by accessing the claim_url and follow the guideline in the url.

In [8]:
# Create a tool set to interact with moltbook

import os
import requests
from langchain_core.tools import tool

MOLTBOOK_API_KEY = userdata.get('MOLTBOOK_API_KEY')
BASE_URL = "https://www.moltbook.com/api/v1"

HEADERS = {
    "Authorization": f"Bearer {MOLTBOOK_API_KEY}",
    "Content-Type": "application/json"
}

# ---------- FEED ----------
@tool
def get_feed(sort: str = "new", limit: int = 10) -> dict:
    """Fetch Moltbook feed."""
    r = requests.get(
        f"{BASE_URL}/feed",
        headers=HEADERS,
        params={"sort": sort, "limit": limit},
        timeout=15
    )
    return r.json()

# ---------- SEARCH ----------
@tool
def search_moltbook(query: str, type: str = "all") -> dict:
    """Semantic search Moltbook posts, comments, agents."""
    r = requests.get(
        f"{BASE_URL}/search",
        headers=HEADERS,
        params={"q": query, "type": type},
        timeout=15
    )
    return r.json()

# ---------- POST ----------
@tool
def create_post(submolt: str, title: str, content: str) -> dict:
    """Create a new text post."""
    payload = {
        "submolt": submolt,
        "title": title,
        "content": content
    }
    r = requests.post(
        f"{BASE_URL}/posts",
        headers=HEADERS,
        json=payload,
        timeout=15
    )
    return r.json()

# ---------- COMMENT ----------
@tool
def comment_post(post_id: str, content: str) -> dict:
    """Comment on a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/comments",
        headers=HEADERS,
        json={"content": content},
        timeout=15
    )
    return r.json()

# ---------- VOTE ----------
@tool
def upvote_post(post_id: str) -> dict:
    """Upvote a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/upvote",
        headers=HEADERS,
        timeout=15
    )
    return r.json()


In [9]:
SYSTEM_PROMPT = """
You are a Moltbook AI agent.

Your purpose:
- Discover valuable AI / ML / agentic system discussions
- Engage thoughtfully and selectively
- NEVER spam
- NEVER repeat content
- Respect rate limits

Rules:
1. Before posting, ALWAYS search Moltbook to avoid duplication.
2. Only comment if you add new insight.
3. Upvote only genuinely useful content.
4. If uncertain, do nothing.
5. Prefer short, clear, professional language.
6. If a human gives an instruction, obey it exactly.

Available tools:
- get_feed
- search_moltbook
- create_post
- comment_post
- upvote_post
"""


# A simple agent to interact with moltbook

In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import ToolMessage
import time
import json
from datetime import datetime
from typing import Any
from google.colab import auth
auth.authenticate_user()

import google.auth
credentials, _ = google.auth.default()

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    vertexai=True,
    project="my-project-5760-485607",
    location="us-central1",
    credentials=credentials,
    temperature=0,
)

def log(section: str, message: str):
    ts = datetime.utcnow().strftime("%H:%M:%S")
    print(f"[{ts}] [{section}] {message}")

def pretty(obj: Any, max_len: int = 800):
    text = json.dumps(obj, indent=2, ensure_ascii=False, default=str)
    return text if len(text) <= max_len else text[:max_len] + "\n...<truncated>"

def moltbook_agent_loop(
    instruction: str | None = None,
    max_turns: int = 8,
    verbose: bool = True,
):
    log("INIT", "Starting Moltbook agent loop")

    tools = [
        get_feed,
        search_moltbook,
        create_post,
        comment_post,
        upvote_post,
    ]

    agent = llm.bind_tools(tools)

    history = [("system", SYSTEM_PROMPT)]

    if instruction:
        history.append(("human", f"Human instruction: {instruction}"))
        log("HUMAN", instruction)
    else:
        history.append(("human", "Perform your Moltbook heartbeat check."))
        log("HEARTBEAT", "No human instruction – autonomous mode")

    # ================================
    # Main agent loop
    # ================================
    for turn in range(1, max_turns + 1):
        log("TURN", f"Turn {turn}/{max_turns} started")
        turn_start = time.time()

        response = agent.invoke(history)
        history.append(response)

        if verbose:
            log("LLM", "Model responded")
            log("LLM.CONTENT", response.content or "<empty>")
            log("LLM.TOOL_CALLS", pretty(response.tool_calls or []))

        # ============================
        # STOP CONDITION
        # ============================
        if not response.tool_calls:
            elapsed = round(time.time() - turn_start, 2)
            log("STOP", f"No tool calls — final answer produced in {elapsed}s")
            return response.content

        # ============================
        # TOOL EXECUTION
        # ============================
        for i, call in enumerate(response.tool_calls, start=1):
            tool_name = call["name"]
            args = call["args"]
            tool_id = call["id"]

            log("TOOL", f"[{i}] Calling `{tool_name}`")
            log("TOOL.ARGS", pretty(args))

            tool_fn = globals().get(tool_name)
            tool_start = time.time()

            try:
                result = tool_fn.invoke(args)
                status = "success"
            except Exception as e:
                result = {"error": str(e)}
                status = "error"

            tool_elapsed = round(time.time() - tool_start, 2)

            log(
                "TOOL.RESULT",
                f"{tool_name} finished ({status}) in {tool_elapsed}s"
            )

            if verbose:
                log("TOOL.OUTPUT", pretty(result))

            history.append(
                ToolMessage(
                    tool_call_id=tool_id,
                    content=str(result),
                )
            )

        turn_elapsed = round(time.time() - turn_start, 2)
        log("TURN", f"Turn {turn} completed in {turn_elapsed}s")

    # ================================
    # MAX TURNS REACHED
    # ================================
    log("STOP", "Max turns reached without final answer")
    return "Agent stopped after reaching max turns."



In [13]:
# You need to complte the tool set so that your agent can find the submolt
moltbook_agent_loop("find submolt named ftec5660")

/tmp/ipython-input-2660/2637156295.py:25: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[18:06:34] [INIT] Starting Moltbook agent loop
[18:06:34] [HUMAN] find submolt named ftec5660
[18:06:34] [TURN] Turn 1/8 started
[18:06:37] [LLM] Model responded
[18:06:37] [LLM.CONTENT] <empty>
[18:06:37] [LLM.TOOL_CALLS] [
  {
    "name": "search_moltbook",
    "args": {
      "query": "ftec5660"
    },
    "id": "3e0a9ff9-f795-42e6-8d90-27331e4d12a5",
    "type": "tool_call"
  }
]
[18:06:37] [TOOL] [1] Calling `search_moltbook`
[18:06:37] [TOOL.ARGS] {
  "query": "ftec5660"
}
[18:06:38] [TOOL.RESULT] search_moltbook finished (success) in 0.35s
[18:06:38] [TOOL.OUTPUT] {
  "success": true,
  "query": "ftec5660",
  "type": "all",
  "results": [
    {
      "id": "fb94de2f-6a69-4105-9118-2c27da9c21df",
      "type": "submolt",
      "title": "FTEC5660",
      "content": "Discussions, notes, and insights for the FTEC5660 course. AI, agents, experiments, and shared learning.",
      "upvotes": 79,
      "downvotes": 0,
      "created_at": "2026-02-03T08:08:50.553Z",
      "relevance": 0.

[{'type': 'text',
  'text': 'I found the submolt named FTEC5660. Its ID is `fb94de2f-6a69-4105-9118-2c27da9c21df`. It is described as "Discussions, notes, and insights for the FTEC5660 course. AI, agents, experiments, and shared learning."',
  'extras': {'signature': 'CpQDAY89a1+J70BVlAXtHYBmkQ5AKUQTZM4juf/w/PFbR7j0CjsqGb5LBFwjUVwSVeXB+f+zjOYYrow49ulqSR/M7NBKozvT3f9rPUiQ/DKGWOQjjiVyo9VhQH2fYlWds0D8UJIB5KrVP1D4t+XSXjFFTIjl9YUh6mj79+LbwVwiBxI60MYgy2NHPdVGtoPcT05S8JCgUGgtsohwqd3f85NNL9DnEcHDHZgc8tCaBoV1M37sYIaxl/75AcXiFUDN/mI/T9lEtw5nWooceVnUeb44IbEFsBUzayJ5BTEbioKkMxTH++J9dNtN/ZsBs7D3GlgXL81NRLLtjkbB5Pg1J1fiIdBFIPD7EjIIe8QPj5G4r8FuAgWBJqJov4dO9/c9/W8BO7VuLTXeYh4vfqh4uQr2F4cd+rFqw0Xr1/khw5b4g8pg47aTqFvEziEd2hqfgmVBM01SnaXFbdv0NCWpk8JKh6+ylP15F8H3LfTta3UhPaur5syyxradi7YoCTJepBWnlpfNE2796Ng0PIh4RQZiLUQD/bc='}}]

#Homework2_Part2 Code

In [26]:
# =====================================================
# Moltbook Social Agent - Homework 2 Part 2
# Complete solution for FTEC5660
# =====================================================

import os
import requests
import time
import json
from datetime import datetime, timezone
from typing import Any, Dict, List, Optional
from google.colab import auth, userdata

# -------------------------
#  获取API密钥
# -------------------------

# 从Colab Secrets获取Moltbook API Key
try:
    MOLTBOOK_API_KEY = userdata.get('MOLTBOOK_API_KEY')
    if not MOLTBOOK_API_KEY:
        raise ValueError("MOLTBOOK_API_KEY not found")
    print("✅ Moltbook API Key loaded successfully")
except Exception as e:
    print(f"❌ Error loading Moltbook API Key: {e}")
    print("Please set MOLTBOOK_API_KEY in Colab secrets (🔑)")
    MOLTBOOK_API_KEY = None

# Moltbook API配置
BASE_URL = "https://www.moltbook.com/api/v1"
HEADERS = {
    "Authorization": f"Bearer {MOLTBOOK_API_KEY}",
    "Content-Type": "application/json"
}


# =====================================================
# 创建完整的Moltbook工具集
# =====================================================

# ---------- 认证工具 ----------
@tool
def verify_authentication() -> dict:
    """Verify that your Moltbook API key is valid and working."""
    try:
        r = requests.get(
            f"{BASE_URL}/agents/me",
            headers=HEADERS,
            timeout=15
        )
        if r.status_code == 200:
            return {"success": True, "data": r.json(), "message": "Authentication successful"}
        else:
            return {"success": False, "error": f"Authentication failed: {r.status_code}"}
    except Exception as e:
        return {"success": False, "error": str(e)}

# ---------- Submolt工具 ----------
@tool
def list_submolts(limit: int = 20) -> dict:
    """List all available submolts on Moltbook."""
    try:
        r = requests.get(
            f"{BASE_URL}/submolts",
            headers=HEADERS,
            params={"limit": limit},
            timeout=15
        )
        return r.json()
    except Exception as e:
        return {"success": False, "error": str(e)}

@tool
def get_submolt(submolt_name: str) -> dict:
    """Get detailed information about a specific submolt."""
    try:
        r = requests.get(
            f"{BASE_URL}/submolts/{submolt_name}",
            headers=HEADERS,
            timeout=15
        )
        return r.json()
    except Exception as e:
        return {"success": False, "error": str(e)}

@tool
def subscribe_to_submolt(submolt_name: str) -> dict:
    """Subscribe to a submolt to follow its content."""
    try:
        r = requests.post(
            f"{BASE_URL}/submolts/{submolt_name}/subscribe",
            headers=HEADERS,
            timeout=15
        )
        return r.json()
    except Exception as e:
        return {"success": False, "error": str(e)}

@tool
def check_subscription(submolt_name: str) -> dict:
    """Check if you are subscribed to a specific submolt."""
    try:
        r = requests.get(
            f"{BASE_URL}/agents/me/subscriptions",
            headers=HEADERS,
            timeout=15
        )
        data = r.json()
        subscriptions = data.get("subscriptions", [])
        is_subscribed = any(s.get("name") == submolt_name for s in subscriptions)
        return {
            "success": True,
            "is_subscribed": is_subscribed,
            "subscriptions": subscriptions
        }
    except Exception as e:
        return {"success": False, "error": str(e)}

# ---------- 帖子工具 ----------
@tool
def get_post(post_id: str) -> dict:
    """Get detailed information about a specific post."""
    try:
        r = requests.get(
            f"{BASE_URL}/posts/{post_id}",
            headers=HEADERS,
            timeout=15
        )
        return r.json()
    except Exception as e:
        return {"success": False, "error": str(e)}

@tool
def upvote_post(post_id: str) -> dict:
    """Upvote a post to show appreciation."""
    try:
        r = requests.post(
            f"{BASE_URL}/posts/{post_id}/upvote",
            headers=HEADERS,
            timeout=15
        )
        return r.json()
    except Exception as e:
        return {"success": False, "error": str(e)}

@tool
def check_if_upvoted(post_id: str) -> dict:
    """Check if you have already upvoted a post."""
    try:
        # Get post details which includes vote information
        r = requests.get(
            f"{BASE_URL}/posts/{post_id}",
            headers=HEADERS,
            timeout=15
        )
        data = r.json()
        # Check if the post has your vote (implementation may vary)
        has_voted = data.get("user_vote") == 1  # 1 typically means upvoted
        return {
            "success": True,
            "has_upvoted": has_voted,
            "post": data
        }
    except Exception as e:
        return {"success": False, "error": str(e)}

@tool
def comment_on_post(post_id: str, content: str) -> dict:
    """Add a thoughtful comment to a post."""
    try:
        r = requests.post(
            f"{BASE_URL}/posts/{post_id}/comments",
            headers=HEADERS,
            json={"content": content},
            timeout=15
        )
        return r.json()
    except Exception as e:
        return {"success": False, "error": str(e)}

@tool
def get_my_comments(limit: int = 10) -> dict:
    """Get your recent comments."""
    try:
        r = requests.get(
            f"{BASE_URL}/agents/me/comments",
            headers=HEADERS,
            params={"limit": limit},
            timeout=15
        )
        return r.json()
    except Exception as e:
        return {"success": False, "error": str(e)}

# ---------- Feed工具 ----------
@tool
def get_feed(sort: str = "new", limit: int = 10) -> dict:
    """Get the main feed of posts."""
    try:
        r = requests.get(
            f"{BASE_URL}/feed",
            headers=HEADERS,
            params={"sort": sort, "limit": limit},
            timeout=15
        )
        return r.json()
    except Exception as e:
        return {"success": False, "error": str(e)}

@tool
def get_submolt_feed(submolt_name: str, sort: str = "new", limit: int = 10) -> dict:
    """Get feed from a specific submolt."""
    try:
        r = requests.get(
            f"{BASE_URL}/submolts/{submolt_name}/feed",
            headers=HEADERS,
            params={"sort": sort, "limit": limit},
            timeout=15
        )
        return r.json()
    except Exception as e:
        return {"success": False, "error": str(e)}

# ---------- 搜索工具 ----------
@tool
def search_moltbook(query: str, type: str = "all") -> dict:
    """Search for posts, comments, or agents."""
    try:
        r = requests.get(
            f"{BASE_URL}/search",
            headers=HEADERS,
            params={"q": query, "type": type},
            timeout=15
        )
        return r.json()
    except Exception as e:
        return {"success": False, "error": str(e)}

# =====================================================
# Agent配置
# =====================================================

# 任务常量
AGENT_NAME = f"Xu_1155245678"
TARGET_SUBMOLT = "ftec5660"
TARGET_POST_ID = "47ff50f3-8255-4dee-87f4-2c3637c7351c"
TARGET_POST_URL = f"https://www.moltbook.com/post/{TARGET_POST_ID}"

# 系统提示
SYSTEM_PROMPT = f"""You are a student named {AGENT_NAME} in the FTEC5660 course. You are participating in the class Moltbook community.

YOUR MISSION:
You must complete the following tasks IN ORDER:

Task 1: Subscribe to submolt /m/{TARGET_SUBMOLT}
Task 2: Upvote the post at {TARGET_POST_URL}
Task 3: Add a thoughtful comment to that post just like a student in this class

RULES:
- NEVER spam - one upvote and one comment only
- Comment must be thoughtful and relevant (2-3 sentences)
- Be professional and helpful
- After completing all tasks, report "MISSION COMPLETE"

AVAILABLE TOOLS:
1. verify_authentication() - Check if your API key works
2. list_submolts() - See all submolts
3. get_submolt(name) - Get submolt details
4. subscribe_to_submolt(name) - Subscribe to a submolt
5. check_subscription(name) - Verify subscription status
6. get_post(post_id) - Get post details
7. upvote_post(post_id) - Upvote a post
8. check_if_upvoted(post_id) - Check if already upvoted
9. comment_on_post(post_id, content) - Add a comment
10. get_my_comments() - See your comments
11. get_feed() - Main feed
12. get_submolt_feed(name) - Submolt-specific feed
13. search_moltbook(query) - Search for content

STRATEGY:
1. First verify authentication
2. Find and subscribe to /m/{TARGET_SUBMOLT}
3. Locate the specific post
4. Upvote it (if not already upvoted)
5. Add a thoughtful comment about AI agents
6. Verify all tasks are complete
7. Report MISSION COMPLETE
"""

# =====================================================
# 日志工具
# =====================================================

def log(section: str, message: str):
    """Pretty logging with timestamps."""
    ts = datetime.now(timezone.utc).strftime("%H:%M:%S")
    print(f"[{ts}] [{section}] {message}")

def pretty_print(obj: Any, max_len: int = 500):
    """Pretty print JSON objects."""
    try:
        text = json.dumps(obj, indent=2, ensure_ascii=False, default=str)
        if len(text) <= max_len:
            return text
        return text[:max_len] + "\n...<truncated>"
    except:
        return str(obj)

# =====================================================
# Agent执行器
# =====================================================

class MoltbookAgent:
    def __init__(self, llm, tools, agent_name: str):
        self.llm = llm.bind_tools(tools)
        self.tools = {tool.name: tool for tool in tools}
        self.agent_name = agent_name
        self.history = []
        self.completed_tasks = set()

    def run(self, max_turns: int = 15) -> str:
        """Run the agent to complete all tasks."""

        log("INIT", f"Starting Moltbook Agent: {self.agent_name}")
        log("TASKS", f"1. Subscribe to /m/{TARGET_SUBMOLT}")
        log("TASKS", f"2. Upvote post {TARGET_POST_ID}")
        log("TASKS", f"3. Comment on post")

        # Initialize conversation
        self.history = [("system", SYSTEM_PROMPT)]
        self.history.append(("human", "Please complete all three tasks in order."))

        for turn in range(1, max_turns + 1):
            log("TURN", f"Turn {turn}/{max_turns}")
            turn_start = time.time()

            try:
                # Get LLM response
                response = self.llm.invoke(self.history)
                self.history.append(response)

                log("LLM", f"Content: {response.content or '[No content]'}")

                # Check if mission complete
                if response.content and "MISSION COMPLETE" in response.content:
                    log("SUCCESS", "✅ All tasks completed successfully!")
                    return response.content

                # Process tool calls
                if response.tool_calls:
                    for tool_call in response.tool_calls:
                        self._execute_tool(tool_call)
                else:
                    # No tool calls, check if we should continue
                    if turn >= max_turns:
                        break

                turn_elapsed = round(time.time() - turn_start, 2)
                log("TURN", f"Turn {turn} completed in {turn_elapsed}s")

                # Small delay to avoid rate limits
                time.sleep(1)

            except Exception as e:
                log("ERROR", f"Error in turn {turn}: {str(e)}")
                return f"Error: {str(e)}"

        log("WARN", "Max turns reached without completion")
        return "Agent stopped: max turns reached"

    def _execute_tool(self, tool_call):
        """Execute a tool call and add result to history."""
        tool_name = tool_call["name"]
        tool_args = tool_call["args"]
        tool_id = tool_call["id"]

        log("TOOL", f"Calling `{tool_name}`")
        log("ARGS", pretty_print(tool_args))

        if tool_name not in self.tools:
            error_msg = f"Tool {tool_name} not found"
            log("ERROR", error_msg)
            self.history.append(
                ToolMessage(
                    tool_call_id=tool_id,
                    content=json.dumps({"error": error_msg})
                )
            )
            return

        tool_start = time.time()
        try:
            result = self.tools[tool_name].invoke(tool_args)
            status = "success"

            # Track task completion
            if tool_name == "subscribe_to_submolt" and tool_args.get("submolt_name") == TARGET_SUBMOLT:
                if result.get("success"):
                    self.completed_tasks.add("subscribe")
                    log("PROGRESS", "✅ Task 1 complete: Subscribed to ftec5660")

            elif tool_name == "upvote_post" and tool_args.get("post_id") == TARGET_POST_ID:
                if result.get("success"):
                    self.completed_tasks.add("upvote")
                    log("PROGRESS", "✅ Task 2 complete: Upvoted post")

            elif tool_name == "comment_on_post" and tool_args.get("post_id") == TARGET_POST_ID:
                if result.get("success"):
                    self.completed_tasks.add("comment")
                    log("PROGRESS", "✅ Task 3 complete: Added comment")

        except Exception as e:
            result = {"error": str(e)}
            status = "error"

        tool_elapsed = round(time.time() - tool_start, 2)
        log("RESULT", f"{tool_name} finished ({status}) in {tool_elapsed}s")
        log("OUTPUT", pretty_print(result))

        # Add result to history
        self.history.append(
            ToolMessage(
                tool_call_id=tool_id,
                content=json.dumps(result, default=str)
            )
        )

# =====================================================
# 运行Agent
# =====================================================

def main():
    """Main execution function."""

    print("\n" + "=" * 80)
    print(f"🚀 MOLTBOOK SOCIAL AGENT - {AGENT_NAME}")
    print("=" * 80)
    print(f"📌 Target Submolt: /m/{TARGET_SUBMOLT}")
    print(f"📌 Target Post: {TARGET_POST_URL}")
    print("=" * 80)

    # Verify API key first
    if not MOLTBOOK_API_KEY:
        print("❌ No Moltbook API key found. Please set MOLTBOOK_API_KEY in Colab secrets.")
        return

    print("\n🔑 Verifying Moltbook authentication...")
    auth_result = verify_authentication.invoke({})
    if not auth_result.get("success"):
        print(f"❌ Authentication failed: {auth_result.get('error')}")
        print("Please check your API key and try again.")
        return
    print("✅ Moltbook authentication successful!")
    print(f"   Agent info: {pretty_print(auth_result.get('data', {}))}")

    # Ask user which mode to run
    print("\n" + "-" * 50)
    print("Select execution mode:")
    print("1. Autonomous Agent (recommended)")

    # Create and run agent
    tools = [
            verify_authentication,
            list_submolts,
            get_submolt,
            subscribe_to_submolt,
            check_subscription,
            get_post,
            upvote_post,
            check_if_upvoted,
            comment_on_post,
            get_my_comments,
            get_feed,
            get_submolt_feed,
            search_moltbook,
        ]

    agent = MoltbookAgent(llm, tools, AGENT_NAME)
    result = agent.run(max_turns=10)

    print("\n" + "=" * 80)
    print("📊 FINAL RESULT")
    print("=" * 80)
    print(result)

    # Show completed tasks
    print("\n✅ Completed tasks:", agent.completed_tasks)
    if len(agent.completed_tasks) == 3:
            print("🎉 All tasks completed successfully!")
    else:
            print(f"⚠️ Only {len(agent.completed_tasks)}/3 tasks completed")
            print("Running manual verification...")
            manual_execution()

# Run the main function
if __name__ == "__main__":
    main()

✅ Moltbook API Key loaded successfully

🚀 MOLTBOOK SOCIAL AGENT - Xu_1155245678
📌 Target Submolt: /m/ftec5660
📌 Target Post: https://www.moltbook.com/post/47ff50f3-8255-4dee-87f4-2c3637c7351c

🔑 Verifying Moltbook authentication...
✅ Moltbook authentication successful!
   Agent info: {
  "success": true,
  "agent": {
    "id": "a2f5b1fa-6ff5-4694-91ae-1ae70663a736",
    "name": "xu_1155245678",
    "display_name": "xu_1155245678",
    "description": "FTEC5660 Homework Agent",
    "karma": 0,
    "follower_count": 0,
    "following_count": 0,
    "posts_count": 0,
    "comments_count": 0,
    "is_verified": false,
    "is_claimed": true,
    "is_active": true,
    "claimed_by": "5bdb990b-8163-474c-b44f-62d8255ada41",
    "created_at": "2026-02-27T17:50:54.444Z",
    "last_act
...<truncated>

--------------------------------------------------
Select execution mode:
1. Autonomous Agent (recommended)
[18:40:06] [INIT] Starting Moltbook Agent: Xu_1155245678
[18:40:06] [TASKS] 1. Subscribe t

Feedback: None


[18:40:24] [LLM] Content: [No content]
[18:40:24] [TURN] Turn 7 completed in 0.52s
[18:40:25] [TURN] Turn 8/10


Feedback: None


[18:40:26] [LLM] Content: [No content]
[18:40:26] [TURN] Turn 8 completed in 1.29s
[18:40:27] [TURN] Turn 9/10


Feedback: None


[18:40:29] [LLM] Content: [No content]
[18:40:29] [TURN] Turn 9 completed in 1.97s
[18:40:30] [TURN] Turn 10/10


Feedback: None


[18:40:31] [LLM] Content: [No content]
[18:40:31] [WARN] Max turns reached without completion

📊 FINAL RESULT
Agent stopped: max turns reached

✅ Completed tasks: {'upvote', 'subscribe', 'comment'}
🎉 All tasks completed successfully!
